In [65]:
import paramiko
import os
import numpy as np
import time
from fabric import Connection

delete_existing = False
upload_list = [1]
success_list = np.zeros(6) #every time a file is successfully uploaded (meaning not exception is thrown)
start_simulation = True

#creation of time based naming string to create unique name if renaming is enabled
result = time.localtime()
time_name = str(result.tm_year) + '_' + str(result.tm_mon) + '_' + str(result.tm_mday) + '_' + str(result.tm_hour) + '_' + str(result.tm_min) + '_' + str(result.tm_sec) + '_'

def upload_HCA(__delete__, __upload__, __time__, __StartSim__):
    #__delete__ : (binary) where True means that if a file exists in the remote machines directory with the name remote_file it will first be deleted else that file will be renamed using the __time__ input
    #__upload__ : (iterable) list that contains the analyzer numbers you would like to upload eg. [2,4,6] will only upload to analyzers 2 and 4 and 6
    #__time__ : (str) unique time based name for renaming created by the time library
    
    __file__ = 'Untitled.ipynb' #you will need to change this to your program name
    
    # Relative pathing to document
    absolutepath = os.path.abspath(__file__)
    parent = os.path.dirname(absolutepath) #returns the absolute path of this file; absolute paths start with the drive like this: "C:\Users\ngulati"
    
    HOST = '192.168.242.119'
    un = 'beacon_test'
    pw = 'beacontest$'
    
    ssh_client = paramiko.SSHClient() #creates SSH object - this is NOT the same as an FTP connection SSH is how we connect to things to send them terminal commands
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy()) #adds a policy to ignore a missing host key
    ssh_client.connect(hostname=HOST,username=un,password=pw) #establishes the connection to the remote PC
    for n in range(1,7,1):
        if n in __upload__:
            local_file = f'capability{n}.json' #filename leveraging f-strings to enable looping over file names (if you choose to have the output of the input file creation in difference folders instead of being name differently you can leverage the f-string in the local path which will require you to change that declaration a bit as it presently uses "os" to do some of the heavy lifting for you)
            remote_file = 'capability.json' #this filename is given to the file transferred meaning it does not implicitly inherit the name of the file from your PC HENCE I have declared this to be capability.json as this is what you want the file to be named in its destination
            remote_path = f'/usr/bin/appHCA{n}/data' #remote path variable leverage f-strings to enable looping over the 6 appHCA folders
            local_path = os.path.join(parent + '\\' + local_file) #created by merging the absolute path to this script with the file name

            stdin, stdout, stderr = ssh_client.exec_command(f"echo '{pw}' | sudo -S chmod 777 /usr/bin/appHCA{n}/data")
            print(f'updated permissions for appHCA{n} folder')

            ftp_client = ssh_client.open_sftp() #open SFTP connection
            ftp_client.chdir(remote_path) #change directory to remote path variable
            print(ftp_client.getcwd()) #print for confirmation of directory change (can be deleted)

            if __delete__ == True:
                try:
                    ftp_client.remove(remote_file)
                    print(f"successfully deleted {remote_file} from {remote_path}")
                except:
                    print(f"unable to delete {remote_file} from {remote_path}")
            else:
                try:
                    ftp_client.rename(remote_file, __time__+remote_file)
                    print(f"successfully renamed {remote_file} to {__time__+remote_file}")
                except:
                    pass
            try:
                ftp_client.put(local_path,remote_file, confirm = True) #file transfer
                ftp_client.close()
                success_list[n-1] = 1
                print(f'uploaded {local_file} to remote PC as {remote_file} into appHCA{n} folder')
            except:
                pass
    try:
        print("Upload Complete, Closing Connection...")
        ssh_client.close()
        print("Connection Closed")
    except:
        print('Error closing connection')

    if __StartSim__ == True:
        
        ##Method 1: using paramiko
        try:
            ssh_client.connect(hostname=HOST,username=un,password=pw)
            stdin, stdout, stderr = ssh_client.exec_command('export DISPLAY=:0.0 && gnome-terminal -- bash -login -c "/home/beacon_test/beacon_automation/jenkins/scripts/startAllHCA_sim.sh -a 1; exec bash"')
            for line in iter(stdout.readline, ""):
                print(line, end="")
            for line in iter(stderr.readline, ""):
                print(line, end="")
            print("Successfully Start HCAs, Closing Connection...")
            ssh_client.close()
            print("Connection Closed")
        except:
            print("Error establishing connection, appHCA not started")

        ##Method 2: Using Transport Channels from Paramkiko (UNTESTED)
#         ssh_client.connect(hostname=HOST,username=un,password=pw)
#         transport = ssh_client.get_transport()
#         channel = transport.open_session()
#         pty = channel.get_pty()
#         shell = ssh_client.invoke_shell()
#         shell.send("cd /home/beacon_test/beacon_automation/jenkins/scripts/")
#         shell.send("./startAllHCA_sim.sh -a 1")
#         channel.exec_command("nohup /home/beacon_test/beacon_automation/jenkins/scripts/startAllHCA_sim.sh -a 1")

        

        ##Method 3 - Using fabric
#         C = Connection(host = HOST, user = un, connect_kwargs = {"password": pw})
#         result = C.run('export DISPLAY=:0.0 && gnome-terminal -- bash -login -c "/home/beacon_test/beacon_automation/jenkins/scripts/startAllHCA_sim.sh -a 1 -g 1; exec bash"')
#         try:
#             result = C.sudo('/home/beacon_test/beacon_automation/jenkins/scripts/startAllHCA_sim.sh -a 1 -g 1', password = pw)
#             print(result)
#         except:
#             print('Execution failed')
#             C.close()
        
upload_HCA(delete_existing, upload_list, time_name, start_simulation)

updated permissions for appHCA1 folder
/usr/bin/appHCA1/data
successfully renamed capability.json to 2021_9_2_17_9_16_capability.json
uploaded capability1.json to remote PC as capability.json into appHCA1 folder
Upload Complete, Closing Connection...
Connection Closed


[sudo] password: Unable to init server: Could not connect: Connection refused


# Option â€œ-eâ€� is deprecated and might be removed in a later version of gnome-terminal.
# Use â€œ-- â€� to terminate the options and put the command line to execute after it.
# Failed to parse arguments: Cannot open display: 
# Option â€œ-eâ€� is deprecated and might be removed in a later version of gnome-terminal.
# Use â€œ-- â€� to terminate the options and put the command line to execute after it.
# Failed to parse arguments: Cannot open display: 


Unable to init server: Could not connect: Connection refused


Execution failed


Socket exception: An existing connection was forcibly closed by the remote host (10054)
Socket exception: An existing connection was forcibly closed by the remote host (10054)
Socket exception: An existing connection was forcibly closed by the remote host (10054)
Socket exception: An existing connection was forcibly closed by the remote host (10054)
Socket exception: An existing connection was forcibly closed by the remote host (10054)
Socket exception: An existing connection was forcibly closed by the remote host (10054)
Socket exception: An existing connection was forcibly closed by the remote host (10054)
Socket exception: An existing connection was forcibly closed by the remote host (10054)
Socket exception: An existing connection was forcibly closed by the remote host (10054)
Socket exception: An existing connection was forcibly closed by the remote host (10054)
Socket exception: An existing connection was forcibly closed by the remote host (10054)
Socket exception: An existing co